In [1]:
import spacetrack.operators as op
from spacetrack import SpaceTrackClient
import pickle
import pandas as pd
import requests, base64
import numpy as np
import os

In [2]:
### Read Username and password from a text file
### txt file format is a single line with username and password sparated by a comma
##
#EXAMPLE####################################################
# username, password
############################################################

with open(r'C:\Users\dk412\Desktop\spacetrackcreds.txt', 'r') as f:
    content = f.read()
st_un = content.split(",")[0].strip()
st_pw = content.split(",")[1].strip()
udl_un = content.split(",")[2].strip()
udl_pw =content.split(",")[3].strip()

output_dir = r'C:\Users\dk412\Desktop\David\Python Projects\RusSat\output'


In [3]:
udl_un, udl_pw, st_un, st_pw

('david.kurtenbach2',
 '$up3rDup3r!98600',
 'dk4120@gmail.com',
 'Sup3rDup3r!98600')

List of all Russian NORAD IDs

In [4]:
st = SpaceTrackClient(identity=f'{st_un}', password= f'{st_pw}')
#st = SpaceTrackClient(identity=unam, password=pword)

## Test Query 
data = st.tle_latest(iter_lines=True, ordinal=1, epoch='>now-30',
                     mean_motion=op.inclusive_range(0.99, 1.01),
                     eccentricity=op.less_than(0.01), format='tle')

with open(os.path.join(output_dir,'tle_latest.txt'), 'w') as fp:
    for line in data:
        fp.write(line + '\n')

In [5]:
rus_sat = st.satcat(country='CIS', current='Y')
rus_sat

[{'INTLDES': '1957-001A',
  'NORAD_CAT_ID': '1',
  'OBJECT_TYPE': 'ROCKET BODY',
  'SATNAME': 'SL-1 R/B',
  'COUNTRY': 'CIS',
  'LAUNCH': '1957-10-04',
  'SITE': 'TTMTR',
  'DECAY': '1957-12-01',
  'PERIOD': '96.19',
  'INCLINATION': '65.10',
  'APOGEE': '938',
  'PERIGEE': '214',
  'COMMENT': None,
  'COMMENTCODE': '4',
  'RCSVALUE': '0',
  'RCS_SIZE': 'LARGE',
  'FILE': '1',
  'LAUNCH_YEAR': '1957',
  'LAUNCH_NUM': '1',
  'LAUNCH_PIECE': 'A',
  'CURRENT': 'Y',
  'OBJECT_NAME': 'SL-1 R/B',
  'OBJECT_ID': '1957-001A',
  'OBJECT_NUMBER': '1'},
 {'INTLDES': '1958-004A',
  'NORAD_CAT_ID': '7',
  'OBJECT_TYPE': 'ROCKET BODY',
  'SATNAME': 'SL-1 R/B',
  'COUNTRY': 'CIS',
  'LAUNCH': '1958-05-15',
  'SITE': 'TTMTR',
  'DECAY': '1958-12-03',
  'PERIOD': '102.74',
  'INCLINATION': '65.14',
  'APOGEE': '1571',
  'PERIGEE': '206',
  'COMMENT': None,
  'COMMENTCODE': None,
  'RCSVALUE': '0',
  'RCS_SIZE': None,
  'FILE': '1',
  'LAUNCH_YEAR': '1958',
  'LAUNCH_NUM': '4',
  'LAUNCH_PIECE': 'A',
  

In [6]:
type(rus_sat), len(rus_sat)

(list, 24924)

In [7]:
# Save sat cat data to pickle
with open(os.path.join(output_dir,'rus_satcat.pkl'), 'wb') as file:
    pickle.dump(rus_sat, file)

In [8]:
cis_norad_ids = []

for e in rus_sat:
    norad_num = int(e.get('NORAD_CAT_ID'))
    cis_norad_ids.append(norad_num)

cis_norad_ids.sort()

Pull TLEs Spacetrack

In [9]:
test_lst = [51511, 37170]
test_lst = [f"{num}" for num in test_lst]

In [10]:
single_query = st.tle(norad_cat_id= 37170, orderby='epoch', limit=None, format='tle').split('\n')

Parse Spacetrack to DF

In [11]:
def parse_line1(line1):
    return {
        'satellite_num': line1[2:7].strip(),
        'intl_des': line1[9:17].strip(),
        'epoch_year': line1[18:20].strip(),
        'epoch_day': line1[20:32].strip(),
        'first_derivative_mean_motion': line1[33:43].strip(),
        'second_derivative_mean_motion': line1[44:52].strip(),
        'bstar_drag': line1[53:61].strip(),
        'ephemeris_type': line1[62].strip(),
        'element_set_number': line1[64:68].strip(),
        'checksum_1': line1[68].strip()
    }

def parse_line2(line2):
    return {
        'satellite_num': line2[2:7].strip(),
        'inclination': line2[8:16].strip(),  
        'right_ascension': line2[17:25].strip(),  
        'eccentricity': line2[26:33].strip(),  
        'argument_of_perigee': line2[34:42].strip(), 
        'mean_anomaly': line2[43:51].strip(),  
        'mean_motion': line2[52:63].strip(),  
        'revolution_number': line2[63:68].strip(),  
        'checksum_2': line2[68].strip()
    }

tle_pairs = [(single_query[i], single_query[i+1]) for i in range(0, len(single_query)-1, 2)]

tle_data = []
for line1, line2 in tle_pairs:
    tle_dict = {}
    tle_dict.update(parse_line1(line1))
    tle_dict.update(parse_line2(line2))
    tle_data.append(tle_dict)

df = pd.DataFrame(tle_data)
df.head()

,satellite_num,intl_des,epoch_year,epoch_day,first_derivative_mean_motion,second_derivative_mean_motion,bstar_drag,ephemeris_type,element_set_number,checksum_1,inclination,right_ascension,eccentricity,argument_of_perigee,mean_anomaly,mean_motion,revolution_number,checksum_2
0,37170,10049A,10,273.79021038,-.00000758,00000-0,00000+0,0,999,5,062.7893,214.4240,7355878,286.7640,031.5428,02.04484077,0,7
1,37170,10049A,10,273.90678870,-.00000739,00000-0,00000+0,0,999,9,062.7853,214.4021,7355693,286.7770,117.3411,02.04447258,0,4
2,37170,10049A,10,273.90678870,-.00000739,00000-0,00000+0,0,999,9,062.7853,214.4021,7355693,286.7770,117.3411,02.04447258,0,4
3,37170,10049A,10,275.22803280,-.00000383,00000-0,00000+0,0,999,6,062.7883,214.2024,7354666,286.7949,009.8165,02.04448226,3,5
4,37170,10049A,10,276.69535476,.00000052,00000-0,00000+0,0,999,9,062.7896,213.9751,7353770,286.8016,009.7826,02.04443616,6,7


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4393 entries, 0 to 4392
Data columns (total 18 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   satellite_num                  4393 non-null   object
 1   intl_des                       4393 non-null   object
 2   epoch_year                     4393 non-null   object
 3   epoch_day                      4393 non-null   object
 4   first_derivative_mean_motion   4393 non-null   object
 5   second_derivative_mean_motion  4393 non-null   object
 6   bstar_drag                     4393 non-null   object
 7   ephemeris_type                 4393 non-null   object
 8   element_set_number             4393 non-null   object
 9   checksum_1                     4393 non-null   object
 10  inclination                    4393 non-null   object
 11  right_ascension                4393 non-null   object
 12  eccentricity                   4393 non-null   object
 13  arg

In [ ]:
import pandas as pd
from datetime import datetime, timedelta

def create_datetime_from_epoch(df, year_col='epoch_year', day_col='epoch_day'):
    base_dates = pd.to_datetime(df[year_col].astype(str) + '-01-01')
    
    datetime_series = base_dates + pd.to_timedelta(df[day_col].astype(float) - 1, unit='D')
    
    return datetime_series

# Example usage
if __name__ == "__main__":
    df['datetime'] = create_datetime_from_epoch(df)
    
    print("\nSample of conversions:")
    print(pd.concat([df[['epoch_year', 'epoch_day', 'datetime']]], axis=1).head())

In [ ]:
df.head()

Savet to Parquet

In [ ]:
df.to_parquet(os.path.join(output_dir,'test_tle_data.parquet'), index=False)

In [ ]:
df_loaded = pd.read_parquet(os.path.join(output_dir,'test_tle_data.parquet'))
df_loaded.head()

UDL Testing

In [ ]:
n = len(cis_norad_ids)
split_size = n // 10  
split_lists = [cis_norad_ids[i:i + split_size] for i in range(0, n, split_size)]

# Print results
for i, sublist in enumerate(split_lists):
    print(f"Split {i+1}: {sublist}")

In [ ]:
shrt_query = '51511,41854'
query = ",".join(map(str,cis_norad_ids)) # Queries Complete 
print(len(query))
query

In [ ]:
basicAuth = "Basic " + base64.b64encode((f"{udl_un}:{udl_pw}").encode('utf-8')).decode("ascii")

#tle_df = pd.DataFrame()

bulk_url = f"https://unifieddatalibrary.com/udl/elset/history/aodr?epoch=%3E2016-08-21T00:00:00.000000Z&satNo={query}&outputFormat=JSON"
his_url = f"https://unifieddatalibrary.com/udl/elset/history?epoch=2016-08-21T00:00:00.000000Z..2021-08-21T00:00:00.000000Z&satNo={query}"
test_url = f"https://unifieddatalibrary.com/udl/elset/history?epoch=%3E2016-08-21T00:00:00.000000Z&satNo={query}"

result = requests.get(test_url, headers={'Authorization':basicAuth}, verify=False)

print(f"Status code: {result.status_code}")
print(f"Response content: {result.text[:200]}")  # Print first 200 chars to see what we got

if result.status_code == 200 and result.text:
    try:
        tle_df = pd.DataFrame(result.json())
        print("DataFrame created successfully")
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON: {e}")
else:
    print(f"Request failed or empty response")

In [ ]:
tle_df.info()

In [ ]:
tle_df['epoch'].max(),tle_df['epoch'].min()

In [ ]:
tle_df['epoch'].max(),tle_df['epoch'].min()

In [ ]:
basicAuth = "Basic " + base64.b64encode((f"{udl_un}:{udl_pw}").encode('utf-8')).decode("ascii")

tle_df = pd.DataFrame()

for e in cis_norad_ids:    
    test_url = f"https://unifieddatalibrary.com/udl/elset/history?epoch=%3E2016-08-21T00:00:00.000000Z&satNo={e}"

    #url = f"https://unifieddatalibrary.com/udl/elset/history/aodr?epoch=2016-08-21T00%3A00%3A00.000000Z&satNo={e}"
    #url = f"https://unifieddatalibrary.com/udl/elset/history/aodr?epoch=2016-08-21T00:00:00.000000Z..2021-08-21T00:00:00.000000Z&satNo=12747,13065,13074,13379,13800,13992,14176,14264,14759,15335&outputFormat=JSON"
    result = requests.get(test_url, headers={'Authorization':basicAuth}, verify=False)
    json_data = result.json()
    if json_data:
        if not isinstance(json_data,list):
            json_data= [json_data]
        df = pd.DataFrame(json_data)
        df['satNo'] = e
        print(f"Sat Num: {e} with shape {df.shape}")
        tle_df = pd.concat([tle_df, df],ignore_index = True)
    else:
        print(f'{e} has nothing')


In [ ]:
tle_df.to_parquet(os.path.join(output_dir,'test_tle_data.parquet'), index=False)

In [ ]:
tle_df['satNo'].unique()

In [ ]:
tle_df['epoch'].sort_values(ascending=True)

In [ ]:
################################################################
#           PROD CODE - UDL
################################################################

import spacetrack.operators as op
from spacetrack import SpaceTrackClient
import pickle
import pandas as pd
import requests, base64
import numpy as np
import os

def udl_tle_to_par(cntry_nm, output_dir, st_un, st_pw, udl_un, udl_pw):
#######################################################################
# Inputs: cntry_nm = 3 letter conuntry code for TLEs you want to pull, example Russia = 'CIS'
#         output_dir = directory to save final pickle and paraquet to
#         st_un, st_pw = SpaceTrack Username (st_un) & password (st_pw)
#         udl_un, udl_pw = UDL Username (udl_un) & password (udl_pw)
# Outputs: {cntry_nm}_satcat.pkl = Satellite Catalog data from Space track
#          {cntry_nm}_tle_data.parquet = Paraquet file containing dataframe of all TLEs
# Notes: Code is set to only query TLEs from February 2012 to current. Date chosen for 10 year look prior to Ukrain invasion
#        To change date modidfy the epoch in url line (see UDL documentation)
#######################################################################

    st = SpaceTrackClient(identity= f"{st_un}", password= f"{st_pw}")
    sat_cat = st.satcat(country=f'{cntry_nm}', current='Y')

    os.makedirs(output_dir, exist_ok=True)

    print("************* Generating Norad ID list ********************")
    with open(os.path.join(output_dir,f'{cntry_nm}_satcat.pkl'), 'wb') as file:
        pickle.dump(sat_cat, file)

    norad_ids = []

    for e in sat_cat:
        norad_num = int(e.get('NORAD_CAT_ID'))
        norad_ids.append(norad_num)

    norad_ids.sort()

    samp_lst = norad_ids[-10:] ########################### TESTING

    basicAuth = "Basic " + base64.b64encode((f"{udl_un}:{udl_pw}").encode('utf-8')).decode("ascii")

    tle_df = pd.DataFrame()

    for e in norad_ids:    
        test_url = f"https://unifieddatalibrary.com/udl/elset/history?epoch=%3E2016-08-21T00:00:00.000000Z&satNo={e}"
        #url = f"https://unifieddatalibrary.com/udl/elset/history/aodr?epoch=2016-08-21T00%3A00%3A00.000000Z&satNo={e}"
        #url = f"https://unifieddatalibrary.com/udl/elset/history/aodr?epoch=2016-08-21T00:00:00.000000Z..2021-08-21T00:00:00.000000Z&satNo=12747,13065,13074,13379,13800,13992,14176,14264,14759,15335&outputFormat=JSON"
        
        result = requests.get(test_url, headers={'Authorization':basicAuth}, verify=False)
        json_data = result.json()
        if json_data:
            if not isinstance(json_data,list):
                json_data= [json_data]
            df = pd.DataFrame(json_data)
            df['satNo'] = e
            print(f"Sat Num: {e} with shape {df.shape}")
            tle_df = pd.concat([tle_df, df],ignore_index = True)
        else:
            print(f'{e} has nothing')

    tle_df.to_parquet(os.path.join(output_dir,f'{cntry_nm}_tle_data.parquet'), index=False)

#with open(r'C:\Users\dk412\Desktop\spacetrackcreds.txt', 'r') as f:
with open('/homes/dkurtenb/projects/russat/spacetrackcreds.txt','r') as f:
    content = f.read()
st_un = content.split(",")[0].strip()
st_pw = content.split(",")[1].strip()
udl_un = content.split(",")[2].strip()
udl_pw =content.split(",")[3].strip()

#output_dir = r'C:\Users\dk412\Desktop\David\Python Projects\RusSat\output'
output_dir = '/homes/dkurtenb/projects/russat/output'
udl_tle_to_par('CIS', output_dir, st_un, st_pw, udl_un, udl_pw)

In [ ]:
with open(r'C:\Users\dk412\Desktop\David\Python Projects\RusSat\output\CIS_satcat.pkl', 'rb') as file:
    data = pickle.load(file)
len(data)

In [ ]:
data

Test HPC Output Parquet

In [ ]:
df_loaded.to_parquet(r'C:\Users\dk412\Desktop\David\Python Projects\RusSat\dataout_HPC\COMPRESSED_CIS_tle_data.parquet', compression='gzip')
df_loaded.shape

In [ ]:
df_loaded.info()

In [ ]:
df_loaded['satNo'].unique()